# Filtering data and getting stats on a specific month

In [1]:
import os, sys
sys.path.insert(0, '../')
from help_func import sql
import pandas as pd
import numpy as np
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import datetime
from collections import Counter
from itertools import chain
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [2]:
subreddit = 'france'
month = 9
year = 2022

# Importing data

In [3]:
comments = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_comments_merged.parquet', engine='pyarrow')
assert len(comments[comments.duplicated(['comment_id'])]) == 0, "Meh, I found some duplicated comments IDs in the dataframe"

posts = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_posts_merged.parquet', engine='pyarrow')
assert len(posts[posts.duplicated(['post_id'])]) == 0, "Meh, I found some duplicated post IDs in the dataframe"

print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 204240 comments
We have 11286 posts


# Filtering on comments published in the specific month

In [4]:
comments = comments[(comments['year_comment'] == year)
    & (comments['month_comment'] == month)]
posts = posts[(posts['year_post'] == year)
    & (posts['month_post'] == month)]
print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 106720 comments
We have 6927 posts


# Creating SQL databases

In [5]:
name_db = 'reddit_analysis'

engine = sql.insert_df_table(f'sqlite:///{name_db}.db', 'posts', posts.reset_index(drop=True))
engine = sql.insert_df_table(f'sqlite:///{name_db}.db', 'comments', comments.reset_index(drop=True))

In [6]:
# connection = engine.connect()

# result = connection.execute("SELECT * FROM posts")
authors_post = sql.execute_query(engine, 'SELECT author_post FROM posts')
authors_post[:3]

[('JeuDeLaVie',), ('Wonderful-Excuse4922',), ('Personal-Thought9453',)]

# Analysis

## Global
### Number of posts
### Number of comments
### Top 3 posts with the highest number of comments (+ links)
### Average number of comments per posts
### Average number of words per comments
### Number of unique authors (posts + comments)

In [7]:
nb_posts = posts['post_id'].nunique()
nb_comments = comments['comment_id'].nunique()
top_com1 = posts.nlargest(3,'nb_comment').iloc[:1]
top_com2 = posts.nlargest(3,'nb_comment').iloc[1:2]
top_com3 = posts.nlargest(3,'nb_comment').iloc[2:3]
avg_comments_posts = round(posts['nb_comment'].mean(), 2)
avg_words_comments = np.array([len(str(comment).split()) for comment in comments['text_comment']]).mean()
nb_active_users = np.unique(comments['author_post'] + comments['author_comment']).size

## Language
### Top 3 words appearing the most in titles
### Top 3 words appearing the most in comments

In [8]:
top_titles_words = pd.Series(' '.join(posts['title_processed']).split()).value_counts()[:10]
top_titles_words

france    320
pas       212
queen     185
«         181
plus      179
»         170
new       133
|         130
ne        127
best      121
dtype: int64

In [9]:
top_comments_words = pd.Series(' '.join(comments['text_processed']).split()).value_counts()[:10]
top_comments_words

pas      70674
c'est    48364
ne       28210
plus     26680
bien     13778
faire    13639
j'ai     12402
qu'il     8162
non       7793
c’est     6855
dtype: int64

## Flairs
### Number of posts per flair
### Posts with the highest number of comments per flair (linked)

In [10]:
nb_posts_flairs = posts.groupby(['flair']).size().sort_values(ascending=False)[:3].reset_index()
biggest_post_flair1 = posts[posts['flair'] == nb_posts_flairs['flair'][0]].nlargest(1,'nb_comment')
biggest_post_flair2 = posts[posts['flair'] == nb_posts_flairs['flair'][1]].nlargest(1,'nb_comment')
biggest_post_flair3 = posts[posts['flair'] == nb_posts_flairs['flair'][2]].nlargest(1,'nb_comment')

## Users average
### Average posts per actif users
### Average comments per users
### Average number of words per users

In [11]:
avg_posts_user = posts.groupby(['author_post']).size().sum()/nb_active_users
avg_comments_user = comments.groupby(['author_comment']).size().mean()
avg_words_user = comments.groupby(['author_comment'])['nb_words_comment'].mean().mean()

## Users records
### Users with the highest number of posts
### Users with the highest number of comments
### Users with the highest number of words
### User with the best vocabulary (highest number of unique words)
### User that removed the most of his posts
### User that wrote the longest comment

In [12]:
highest_nb_posts_user_query = """
SELECT author_post,
nb_posts
FROM
(SELECT author_post,
COUNT(DISTINCT post_id) as nb_posts
FROM posts
GROUP BY author_post)
ORDER BY nb_posts DESC
LIMIT 5
"""
highest_nb_posts_user = sql.execute_query(engine, highest_nb_posts_user_query)
highest_nb_posts_user

[('latestasianews', 2340),
 ('RIFTV_news', 220),
 ('Hellvis_50s', 57),
 ('Fearless-Cricket3297', 53),
 ('FrankMaleir', 50)]

In [13]:
highest_nb_comments_user_query = """
SELECT author_comment,
nb_comments
FROM
(SELECT author_comment,
COUNT(DISTINCT comment_id) as nb_comments
FROM comments
WHERE author_comment != '[deleted]'
GROUP BY author_comment)
ORDER BY nb_comments DESC
LIMIT 5
"""
highest_nb_comments_user = sql.execute_query(engine, highest_nb_comments_user_query)
highest_nb_comments_user

[('morinl', 1033),
 ('AutoModerator', 713),
 ('anyatrans', 484),
 ('Elegant-Variety-7482', 427),
 ('Irkam', 410)]

In [14]:
highest_nb_words_user = comments.groupby(['author_comment'])['nb_words_comment'].sum().reset_index().sort_values('nb_words_comment', ascending=False).reset_index(drop=True)[:5]

In [15]:
comments.columns

Index(['post_id', 'title', 'text_post', 'url', 'author_post', 'permalink_post',
       'flair', 'author_comment', 'comment_id', 'text_comment',
       'parent_comment_id', 'permalink_comment', 'year_comment',
       'month_comment', 'day_comment', 'year_post', 'month_post', 'day_post',
       'text_processed', 'title_processed', 'sentiment_num', 'sentiment_cat',
       'nb_words_post', 'nb_words_comment'],
      dtype='object')

In [36]:
def nb_unique_words(r) -> int:
    """
    Count unique number of words in a string of a row
    """
    comment = r.text_comment.lower()
    comment = comment.replace("."," ")
    comment = comment.replace(","," ")
    comment = comment.replace(":"," ")
    comment = comment.replace(";"," ")
    comment = comment.replace("?"," ")
    comment = comment.replace(r'\s+|\\n', ' ') 
    words = comment.split(" ")
    unique_words = []
    nb_unique_words = 0
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
            nb_unique_words += 1
    return nb_unique_words

In [38]:
comments_concat_df = comments[comments['text_comment'].str.len() > 0].groupby(['author_comment'], as_index=False).agg({'text_comment': ' '.join})
comments_concat_df['nb_unique_words'] = comments_concat_df.apply(nb_unique_words, axis = 1)
highest_vocabulary_user = comments_concat_df.sort_values('nb_unique_words', ascending=False)[:1]
highest_vocabulary_user

,author_comment,text_comment,nb_unique_words
880,Bandolinho2,C'est même un peu la marque de fabrique de l'é...,12478


In [60]:
comments.columns

Index(['post_id', 'title', 'text_post', 'url', 'author_post', 'permalink_post',
       'flair', 'author_comment', 'comment_id', 'text_comment',
       'parent_comment_id', 'permalink_comment', 'year_comment',
       'month_comment', 'day_comment', 'year_post', 'month_post', 'day_post',
       'text_processed', 'title_processed', 'sentiment_num', 'sentiment_cat',
       'nb_words_post', 'nb_words_comment', 'len_comments', 'len_comment'],
      dtype='object')

In [63]:
comments['len_comment'] = comments['text_comment'].str.len()
longest_comment_user = comments[['author_comment', 'len_comment','permalink_comment']].sort_values('len_comment', ascending=False)[:1]
longest_comment_user

,author_comment,len_comment,permalink_comment
45372,GreyArrowMonkey,9973.0,/r/france/comments/xdhbme/meilleur_combo_banqu...


In [64]:
print(longest_comment_user['permalink_comment'].values)

['/r/france/comments/xdhbme/meilleur_combo_banques_best_free_bank_combo/ioaxarb/']
